# Python interaction

In this page I will show you how to deploy and interact with a Postgres database in a Docker container.

# Python from container

For me it's common task to build interaption with container which handle python program.

So in this section I want to show you how to build such an interaction.

In the following list, some important files for this example are mentioned:

* Python container:

    - [Dockerfile](python_interaction/python_container/dockerfile) for python container;
    - [requrements.txt](python_interaction/python_container/requirements.txt) for this container;
    - [script.py](python_interaction/python_container/script.py) for this container;


* Postgres container:

    - [sql script](python_interaction/postgres_container/create_table.sql) that creates experiment tables in the database.

## Python image

You need to create a Python image.

In [19]:
%%bash
docker build -t pg_example_python python_interaction/python_container &> /dev/null

## Start containers

You should share the same net between containers, in the following example which is completed by `test_project_net`.

In [20]:
%%bash
# network
docker network create test_project_net &> /dev/null
# postgres
docker run --rm -d\
    --name pg_example_posgres_cont\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    --net=test_project_net\
    -v ./python_interaction/create_table.sql:/docker-entrypoint-initdb.d/create_table.sql\
    postgres:15.4 &> /dev/null
# python
docker run --rm -itd\
    --name pg_example_python_cont\
    --net=test_project_net\
    pg_example_python &> /dev/null

## Executing python

In the next cell, we run the Python script in the Python container. The script simply adds some random values to the `main_table` from the **ohter container**. The message "Adding records is done!" indicates that the Python program was executed normally.

In [22]:
%%bash
docker exec pg_example_python_cont bash -c "python3 script.py"

Adding records is done!


## Check the table

So now, to make sure we have done everything right, let us select values from the created table.

In [24]:
%%bash
docker exec pg_example_posgres_cont bash -c \
"psql -U docker_app -d docker_app_db -c \"SELECT * FROM main_table;\""

 id |         text         
----+----------------------
 0  | sroybaubnkkggzuegofw
 1  | afwgyapddpvuukdpkxxc
 2  | lfxohpbhhsdcxfiiqyfp
 3  | lfqwakbcgxefmnwsnixu
 4  | glmxqjoovbyvwwphfcgd
 5  | cagjujihvyntzykigrar
 6  | tagrodfgevcqzeoyrdyt
 7  | ngjgbkkppcvcbspypvsv
 8  | vtyxgkaouosrkszkfwtr
 9  | lwcxnrsrvazombgaqwuo
 10 | dgxyzzjyuhwsotxabdbo
 11 | ahdmlqgmauydtcxmrsrp
 12 | oedpcbffjnblbizucezc
 13 | ijoxobrthmytpirqijbu
 14 | ipunrnsjrrohcwuukzsy
 15 | eprawbrdpvhrmwhwhnrk
 16 | yhlcgjwtuoqcpcbkaify
 17 | eityjwmjonpfvvqarrcq
 18 | rablibcsibalhbkqoynd
 19 | xbnimzepzzbvjqwygiam
(20 rows)



## Stop containers

In [25]:
%%bash
docker stop pg_example_posgres_cont pg_example_python_cont &> /dev/null
docker network rm test_project_net &> /dev/null

## Clear python image

To avoid creating a lot of rubbish in docker images, you should delete the container created for this example.

In [26]:
%%bash
docker rmi pg_example_python &> /dev/null